<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg?__hstc=259489365.a667dfafcfa0169c8aee4178d115dc81.1733501603539.1733501603539.1733501603539.1&__hssc=259489365.1.1733501603539&__hsfp=3822854628&submissionGuid=381a0676-8f38-437b-96f2-fc10875658df#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing a LangGraph Application built on Google Agent Engine</h1>


This notebook is adapted from Google's "[Building and Deploying a LangGraph Application with Agent Engine in Vertex AI](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/tutorial_langgraph.ipynb)"

| | |
|-|-|
| Original Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) is a managed service that helps you to build and deploy agent frameworks. [LangGraph](https://langchain-ai.github.io/langgraph/) is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

This notebook demonstrates how to build, deploy, and test a simple LangGraph application using [Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) in Vertex AI. You'll learn how to combine LangGraph's workflow orchestration with the scalability of Vertex AI, which enables you to build custom generative AI applications.

Note that the approach used in this notebook defines a [custom application template in Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/customize), which can be extended to LangChain or other orchestration frameworks. If just want to use Agent Engine to build agentic generative AI applications, refer to the documentation for [developing with the LangChain template in Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/overview).

This notebook covers the following steps:

- **Define Tools**: Create custom Python functions to act as tools your AI application can use.
- **Define Router**: Set up routing logic to control conversation flow and tool selection.
- **Build a LangGraph Application**: Structure your application using LangGraph, including the Gemini model and custom tools that you define.
- **Local Testing**: Test your LangGraph application locally to ensure functionality.
- **Deploying to Vertex AI**: Seamlessly deploy your LangGraph application to Agent Engine for scalable execution.
- **Remote Testing**: Interact with your deployed application through Vertex AI, testing its functionality in a production-like environment.
- **Cleaning Up Resources**: Delete your deployed application on Vertex AI to avoid incurring unnecessary charges.

By the end of this notebook, you'll have the skills and knowledge to build and deploy your own custom generative AI applications using LangGraph, Agent Engine, and Vertex AI.

## Get started

### Install Vertex AI SDK and other required packages

In [1]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]==1.87.0" \
    cloudpickle==3.0.0 \
    pydantic==2.11.2 \
    langgraph==0.2.76 \
    httpx \
    "arize-phoenix-otel>=0.9.0" \
    "arize-phoenix>=8.26.3" \
    "openinference-instrumentation-langchain>=0.1.4"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.60.2 requires httpx<0.28.0,>=0.23.0, but you have httpx 0.28.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
PROJECT_ID = "sandbox-455622"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://agents-experiments"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Set Arize Phoenix 🐦‍🔥 Env Variables

The following env variables will allow you to connect to an online instance of Arize Phoenix. You can get an API key on the [Phoenix website](https://app.phoenix.arize.com).

If you'd prefer to self-host Phoenix, please see [instructions for self-hosting](https://docs.arize.com/phoenix/deployment). The Cloud and Self-hosted versions are functionally identical.

In [2]:
import os
from getpass import getpass

from dotenv import load_dotenv

load_dotenv()

# Change the following line if you're self-hosting
if os.getenv("PHOENIX_COLLECTOR_ENDPOINT") is None:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

# Remove the following lines if you're self-hosting
if os.getenv("PHOENIX_API_KEY") is None:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
    os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"

## Building and deploying a LangGraph app on Agent Engine

In the following sections, we'll walk through the process of building and deploying a LangGraph application using Agent Engine in Vertex AI.

### Import libraries

Import the necessary Python libraries. These libraries provide the tools we need to interact with LangGraph, Vertex AI, and other components of our application.

In [3]:
from typing import Literal

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode

### Define tools

You'll start by defining the a tool for your LangGraph application. You'll define a custom Python function that act as tools in our agentic application.

In this case, we'll define a simple tool that returns a product description based on the product that the user asks about. In reality, you can write functions to call APIs, query databases, or anything other tasks that you might want your agent to be able to use.

In [4]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "coffee": "A rich, aromatic blend of ethically sourced coffee beans.",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")

### Define router

Then, you'll define a router to control the flow of the conversation, determining which tool to use based on user input or the state of the interaction. Here we'll use a simple router setup, and you can customize the behavior of your router to handle multiple tools, custom logic, or multi-agent workflows.

In [5]:
def router(state: list[BaseMessage]) -> Literal["get_product_details", "__end__"]:
    """Initiates product details retrieval if the user asks for a product."""
    # Get the tool_calls from the last message in the conversation history.
    tool_calls = state[-1].tool_calls
    # If there are any tool_calls
    if len(tool_calls):
        # Return the name of the tool to be called
        return "get_product_details"
    else:
        # End the conversation flow.
        return "__end__"

### Define LangGraph application

Now you'll bring everything together to define your LangGraph application as a custom template in Agent Engine.

This application will use the tool and router that you just defined. LangGraph provides a powerful way to structure these interactions and leverage the capabilities of LLMs.

In [6]:
class SimpleLangGraphApp:
    def __init__(self, project: str, location: str) -> None:
        self.project_id = project
        self.location = location

    # The set_up method is used to define application initialization logic
    def set_up(self) -> None:
        # Phoenix code begins
        from phoenix.otel import register

        register(
            project_name="google-agent-evaluation-langgraph",  # name this to whatever you would like
            auto_instrument=True,  # this will automatically call all openinference libraries (e.g. openinference-instrumentation-langchain)
            endpoint=os.getenv("PHOENIX_COLLECTOR_ENDPOINT") + "/v1/traces",
        )
        # Phoenix code ends

        model = ChatVertexAI(model="gemini-2.0-flash")

        builder = MessageGraph()

        model_with_tools = model.bind_tools([get_product_details])
        builder.add_node("tools", model_with_tools)

        tool_node = ToolNode([get_product_details])
        builder.add_node("get_product_details", tool_node)
        builder.add_edge("get_product_details", END)

        builder.set_entry_point("tools")
        builder.add_conditional_edges("tools", router)

        self.runnable = builder.compile()

    # The query method will be used to send inputs to the agent
    def query(self, message: str):
        """Query the application.

        Args:
            message: The user message.

        Returns:
            str: The LLM response.
        """
        chat_history = self.runnable.invoke(HumanMessage(message))

        return chat_history[-1].content

### Local testing

In this section, you'll test your LangGraph app locally before deploying it to ensure that it behaves as expected before deployment.

In [7]:
agent = SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION)
agent.set_up()

/Users/jgilhuly/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jgilhuly/.local/lib/python3.11/site-packages/phoenix/otel/otel.py:301: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")
Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: google-agent-evaluation-langgraph
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/jg-test/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [8]:
agent.query(message="Get product details for shoes")

'High-performance running shoes designed for comfort, support, and speed.'

In [9]:
agent.query(message="Get product details for coffee")

'A rich, aromatic blend of ethically sourced coffee beans.'

In [10]:
agent.query(message="Get product details for smartphone")

'A cutting-edge smartphone with advanced camera features and lightning-fast processing.'

In [11]:
# Ask a question that cannot be answered using the defined tools
agent.query(message="Tell me about the weather")

'I am sorry, I cannot provide weather information. I can only provide product details.\n'

## Evaluate Agent

In [16]:
import phoenix as px
from phoenix.trace.dsl import SpanQuery

query = (
    SpanQuery()
    .where(
        "span_kind == 'LLM'",
    )
    .select(
        input="input.value",
        output="llm.output_messages",
    )
)

# The Phoenix Client can take this query and return the dataframe.
eval_dataframe = px.Client().query_spans(query, project_name="google-agent-evaluation-langgraph")

In [26]:
# Extract tool calls and parameters from the output
def extract_tool_call(output):
    if not output or not isinstance(output, list) or len(output) == 0:
        return None

    # Check if there's a function call in the first message
    message = output[0].get("message", {})
    if "function_call_name" in message:
        return message.get("function_call_name")
    return None


def extract_tool_parameters(output):
    if not output or not isinstance(output, list) or len(output) == 0:
        return None

    # Check if there's a function call in the first message
    message = output[0].get("message", {})
    if "function_call_arguments_json" in message:
        return message.get("function_call_arguments_json")
    return None


# Add new columns for tool calls and parameters
eval_dataframe["tool_call"] = eval_dataframe["output"].apply(extract_tool_call)
eval_dataframe["tool_parameters"] = eval_dataframe["output"].apply(extract_tool_parameters)

# Display the first few rows to verify
eval_dataframe.head()

,input,output,tool_call,tool_parameters
context.span_id,,,,
80f514975d0b10b4,"{""messages"": [[{""lc"": 1, ""type"": ""constructor""...","[{'message': {'role': 'assistant', 'function_c...",get_product_details,"{""product_name"": ""shoes""}"
62c2200c01451f32,"{""messages"": [[{""lc"": 1, ""type"": ""constructor""...","[{'message': {'role': 'assistant', 'function_c...",get_product_details,"{""product_name"": ""coffee""}"
a0629a4962c45b7c,"{""messages"": [[{""lc"": 1, ""type"": ""constructor""...","[{'message': {'role': 'assistant', 'content': ...",None,None
007139da9ce7bcb0,"{""messages"": [[{""lc"": 1, ""type"": ""constructor""...","[{'message': {'role': 'assistant', 'function_c...",get_product_details,"{""product_name"": ""smartphone""}"


In [39]:
import pandas as pd

# Create a list to store the predicted trajectories
predicted_trajectories = []

# Iterate through each row in the eval_dataframe
for _, row in eval_dataframe.iterrows():
    trajectory = []

    # Only add to trajectory if there's a tool call
    if row["tool_call"] is not None and row["tool_parameters"] is not None:
        trajectory.append({"tool_name": row["tool_call"], "tool_input": row["tool_parameters"]})

    predicted_trajectories.append(trajectory)

# For this example, we'll use the same trajectories as reference
# In a real scenario, you would have actual reference data
reference_trajectories = predicted_trajectories.copy()

# Create the evaluation dataset
eval_dataset = pd.DataFrame(
    {
        "predicted_trajectory": predicted_trajectories,
        "reference_trajectory": reference_trajectories,
        "prompt": eval_dataframe["input"],  # Keep the input column
    }
)

# Display the first few rows to verify
eval_dataset.head()

,predicted_trajectory,reference_trajectory,prompt
context.span_id,,,
80f514975d0b10b4,"[{'tool_name': 'get_product_details', 'tool_in...","[{'tool_name': 'get_product_details', 'tool_in...","{""messages"": [[{""lc"": 1, ""type"": ""constructor""..."
62c2200c01451f32,"[{'tool_name': 'get_product_details', 'tool_in...","[{'tool_name': 'get_product_details', 'tool_in...","{""messages"": [[{""lc"": 1, ""type"": ""constructor""..."
a0629a4962c45b7c,[],[],"{""messages"": [[{""lc"": 1, ""type"": ""constructor""..."
007139da9ce7bcb0,"[{'tool_name': 'get_product_details', 'tool_in...","[{'tool_name': 'get_product_details', 'tool_in...","{""messages"": [[{""lc"": 1, ""type"": ""constructor""..."


In [40]:
from vertexai.preview.evaluation import PointwiseMetric, PointwiseMetricPromptTemplate

response_follows_trajectory_prompt_template = PointwiseMetricPromptTemplate(
    criteria={
        "Follows trajectory": (
            "Evaluate whether the agent's response logically follows from the "
            "sequence of actions it took. Consider these sub-points:\n"
            "  - Does the response reflect the information gathered during the trajectory?\n"
            "  - Is the response consistent with the goals and constraints of the task?\n"
            "  - Are there any unexpected or illogical jumps in reasoning?\n"
            "Provide specific examples from the trajectory and response to support your evaluation."
        )
    },
    rating_rubric={
        "1": "Follows trajectory",
        "0": "Does not follow trajectory",
    },
    input_variables=["prompt", "predicted_trajectory"],
)

response_follows_trajectory_metric = PointwiseMetric(
    metric="response_follows_trajectory",
    metric_prompt_template=response_follows_trajectory_prompt_template,
)

In [41]:
from vertexai.preview.evaluation import EvalTask

eval_task = EvalTask(dataset=eval_dataset, metrics=[response_follows_trajectory_metric])
eval_result = eval_task.evaluate(runnable=agent)

  0%|          | 0/4 [00:00<?, ?it/s]

When a `runnable` is provided, trajectory isgenerated dynamically by the runnable, so the pre-existing `response` column provided in the evaluation dataset is not used.
When a `runnable` is provided, trajectory isgenerated dynamically by the runnable, so the pre-existing `response` column provided in the evaluation dataset is not used.
When a `runnable` is provided, trajectory isgenerated dynamically by the runnable, so the pre-existing `response` column provided in the evaluation dataset is not used.
When a `runnable` is provided, trajectory isgenerated dynamically by the runnable, so the pre-existing `response` column provided in the evaluation dataset is not used.


100%|██████████| 4/4 [00:00<00:00, 1443.95it/s]

All 4 responses are successfully generated from the runnable.
Computing metrics with a total of 4 Vertex Gen AI Evaluation Service API requests.



  0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'decode'

In [ ]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(SpanEvaluations(eval_name="Trajectory Eval", dataframe=eval_result))